In [12]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import datetime
from pyspark.sql import types
from collections import namedtuple
import pandas as pd

In [3]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/02 18:56:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/02 18:56:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark.sparkContext.uiWebUrl

'http://dtc-de.asia-southeast1-b.c.de-dtc-375915.internal:4041'

In [5]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [13]:
columns = ["VendorID", "lpep_pickup_datetime", "PULocationID", "DOLocationID", "trip_distance"]
duration_rdd = ( 
    df_green
    .select(columns)
    .rdd 
)


In [16]:
def model_predict(df):
    y_pred = df.trip_distance * 5

    return y_pred

In [18]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)

    df["predicted_duration"] = predictions

    for row in df.itertuples():
        yield row


In [21]:
duration_rdd.mapPartitions(apply_model_in_batch).toDF().show()

+-----+--------+--------------------+------------+------------+-------------+------------------+
|Index|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|predicted_duration|
+-----+--------+--------------------+------------+------------+-------------+------------------+
|    0|     2.0|                  {}|          74|         130|        12.77|63.849999999999994|
|    1|     NaN|                  {}|          67|          39|          8.0|              40.0|
|    2|     2.0|                  {}|         260|         157|         1.27|              6.35|
|    3|     2.0|                  {}|          82|          83|         1.25|              6.25|
|    4|     2.0|                  {}|         166|          42|         1.84| 9.200000000000001|
|    5|     2.0|                  {}|         179|         223|         0.76|               3.8|
|    6|     2.0|                  {}|          41|         237|         3.32|16.599999999999998|
|    7|     2.0|              